# Some Theory and Imports

H2O AutoML is an automated machine learning meta-algorithm that is part of the H2O software library. (It shold not be confused with H2O DriverlessAI, which is a commercial product and built from an entirely different code base.) H2O’s AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit. Stacked Ensembles – one based on all previously trained models, another one on the best model of each family – will be automatically trained on collections of individual models to produce highly predictive ensemble models which, in most cases, will be the top performing models in the AutoML Leaderboard.

Automated machine learning (AutoML) is the process of automating the end-to-end process of applying machine learning to real-world problems. AutoML makes machine learning available in a true sense, even to people with no major expertise in this field.

Advantages
The advantages of AutoML can be summed up in three major points:

Increases productivity by automating repetitive tasks. This enables a data scientist to focus more on the problem rather than the models.
Automating the ML pipeline also helps to avoid errors that might creep in manually.
Ultimately, AutoML is a step towards democratizing machine learning by making the power of ML accessible to everybody.
H2O AutoML
H2O’s AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit. Stacked Ensembles – one based on all previously trained models, another one on the best model of each family – will be automatically trained on collections of individual models to produce highly predictive ensemble models which, in most cases, will be the top performing models in the AutoML Leaderboard.

Properties of H2O AutoML

Basic data pre-processing (as in all H2O algos).

Trains a random grid of GBMs, DNNs, GLMs, etc. using a carefully chosen hyper-parameter space.

Individual models are tuned using cross-validation.

Two Stacked Ensembles are trained (“All Models” ensemble & a lightweight “Best of Family” ensemble).

Returns a sorted “Leaderboard” of all models. All models can be easily exported to production.

In [ ]:
import os
import sys
import time
import random
import logging
import typing as tp
from pathlib import Path
from contextlib import contextmanager

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error, mean_squared_error

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoost, Pool

%matplotlib inline

In [ ]:
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "tabular-playground-series-jan-2021"
WORK = ROOT / "working"

for path in DATA.iterdir():
    print(path.name)

In [ ]:
# train = pd.read_csv(DATA / "train.csv")
# test = pd.read_csv(DATA / "test.csv")
# smpl_sub = pd.read_csv(DATA / "sample_submission.csv")
# print("train: {}, test: {}, sample sub: {}".format(
#     train.shape, test.shape, smpl_sub.shape
# ))

In [ ]:
import os
print(os.listdir("../input/tabular-playground-series-jan-2021/"))

### Import the h2o Python module and H2OAutoML class and initialize a local H2O cluster

In [ ]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

# Get Data and Initialize H2O Cluster

In [ ]:
train = h2o.import_file("../input/tabular-playground-series-jan-2021/train.csv")
test = h2o.import_file("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
train.head()

In [ ]:
x = test.columns[1:]
y = 'target'

In [ ]:
# train[y] = train[y].astype(int)

In [ ]:
# For binary classification, response should be a factor
# train[y] = train[y].asfactor()

## Run AutoML
Run AutoML, stopping after around 1 hour. The max_runtime_secs argument provides a way to limit the AutoML run by time. When using a time-limited stopping criterion, the number of models train will vary between runs. If different hardware is used or even if the same machine is used but the available compute resources on that machine are not the same between runs, then AutoML may be able to train more models on one run vs another.

In [ ]:
aml = H2OAutoML(max_runtime_secs = 50, seed = 1, project_name = "lb_frame")
# max_models=200
aml.train(x = x, y = y, training_frame = train)
# change max run time secs to at least 1 hour or more

## Leaderboard
Next, we will view the AutoML Leaderboard. Since we specified a leaderboard_frame in the H2OAutoML.train() method for scoring and ranking the models, the AutoML leaderboard uses the performance on this data to rank the models.

A default performance metric for each machine learning task (binary classification, multiclass classification, regression) is specified internally and the leaderboard will be sorted by that metric. In the case of regression, the default ranking metric is mean residual deviance. In the future, the user will be able to specify any of the H2O metrics so that different metrics can be used to generate rankings on the leaderboard.

In [ ]:
lb = aml.leaderboard
lb.head() 

In [ ]:
lb.head(rows=lb.nrows) 

In [ ]:
aml.leader

# Train, Predict, Explore for more

In [ ]:
preds = aml.predict(test)

## Ensemble Exploration
To understand how the ensemble works, let's take a peek inside the Stacked Ensemble "All Models" model. The "All Models" ensemble is an ensemble of all of the individual models in the AutoML run. This is often the top performing model on the leaderboard.

In [ ]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
metalearner.coef_norm()

Examine the variable importance of the metalearner (combiner) algorithm in the ensemble. This shows us how much each base learner is contributing to the ensemble.

In [ ]:
metalearner.std_coef_plot()


In [ ]:
pred = aml.leader.predict(test)
pred.head()

## Save Leader Model
You can also save and download your model and use it for deploying it to productiont.

In [ ]:
h2o.save_model(aml.leader, path = "./product_backorders_model_bin")

# Submit

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')
sample_submission.shape

In [ ]:
sample_submission

In [ ]:
sample_submission['target'] = pred.as_data_frame().values

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('h2o_automl_submission_5.csv', index=False)

In [ ]:
h2o.save_model(aml.leader, path = "submission1.csv")

In [ ]:
sample_submission.head()

for theory: [credits](https://www.kaggle.com/parulpandey/automating-the-ml-workflow-with-h2o-automl)

## Appreciated?